In [1]:
# Librerias utiles
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import plotly
import seaborn as sns 
import plotly.express as px
import missingno as msno
import plotly.io as pio
import plotly.graph_objects as go
from mapsmx import MapsMX
import geopandas
import pyproj
import json
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
sns.set_theme()

In [2]:
app = dash.Dash()

## Load dfs and clean

In [3]:
feminicidios = pd.read_csv('../BD-sucias/feminicidios-corregida.csv', low_memory=False)
feminicidios.fecha_recepcion = pd.to_datetime(feminicidios.fecha_recepcion, format='%d/%m/%Y', errors='ignore')
feminicidios['año_recepcion'] = feminicidios['fecha_recepcion'].dt.year
nas = feminicidios[(feminicidios.TipoRelacion.isna()==True)|(feminicidios.TipoRelacion=='Seleccione')].index
feminicidios.loc[nas,'TipoRelacion'] = 'Desconocido'
com = feminicidios[(feminicidios.TipoRelacion=='En la comunidad')].index
feminicidios.loc[com,'TipoRelacion'] = 'Comunidad'

In [4]:
servicios = pd.read_csv('../BD-sucias/servicios_11_OCTUBRE_2021.csv', low_memory=False)

In [5]:
victimas = pd.read_csv('reportes/reporte_semujeres_11_OCTUBRE_2021.csv',low_memory=False, dtype={'pk_perfil_agresor': 'object','num_hijos':'int32'}, parse_dates=['fecha_recepcion','fecha_hechos'])

In [6]:
victimas = pd.concat([victimas, victimas.tipos.str.get_dummies(sep=',')], axis=1)
victimas['Feminicida']=0
fems = victimas[victimas['descripcion_otro_tipos']=='FEMINICIDA'].index
victimas.loc[fems,'Feminicida'] = 1
victimas.loc[fems,'Otro'] = 0

In [7]:
validez = victimas[victimas.escolaridad=='Estudios que no requieren validéz oficial'].index
tecnica = victimas[victimas.escolaridad=='Carrera técnica comercial'].index
victimas.loc[validez,'escolaridad']='Sin validez'
victimas.loc[tecnica,'escolaridad']='Carrera técnica'

In [8]:
feminicidios = feminicidios.merge(victimas[['fk_euv','Edad Agresor']], left_on='fk_euv', right_on='fk_euv', how='left')
feminicidios = feminicidios.drop_duplicates(keep='last')
feminicidios = feminicidios.reset_index(drop=True)
fem_agrav = feminicidios[feminicidios.Sentencia.isin(['FEMINICIDIO AGRAVADO','FEMINICIDIO'])].index
feminicidios.loc[fem_agrav,'Sentencia']='FEMINICIDIO'
fem_agrav = feminicidios[feminicidios.Sentencia.isin(['FEMINICIDIO AGRAVADO EN GRADO DE TENTATIVA','FEMINICIDIO EN GRADO DE TENTATIVA'])].index
feminicidios.loc[fem_agrav,'Sentencia']='TENTATIVA DE FEMINICIDIO'

In [9]:
subtipo = pd.read_csv('../BD-sucias/subtipo_01_OCTUBRE_2021.csv', low_memory=False)
subtipo.fecha_recepcion = pd.to_datetime(subtipo.fecha_recepcion, format='%d/%m/%Y', errors='ignore')
subtipo.fecha_hechos = pd.to_datetime(subtipo.fecha_hechos, format='%d/%m/%Y', errors='ignore')
subtipo = subtipo.dropna()
selecciones = subtipo[subtipo.SubtipoOrd=='Seleccione'].index
subtipo.loc[selecciones, 'SubtipoOrd'] = 'No especificado'

In [10]:
state = MapsMX().get_geo('state')
muns = MapsMX().get_geo('municipality')
yuc = muns[muns['cve_ent']=='31']

# Dashboard

## Layout

In [11]:
app.layout = html.Div(
    children=[
        html.H1(children="SEMUJERES TABLERO",),
        html.P(
            children="Texto de prueba",
        ),
        html.Div(
            children=[
                html.Div(
                    children=[
                        html.Div(children="Dependencia de recepcion", className="menu-title"),
                        dcc.Dropdown(
                            id="dependencia-filter",
                            options=[
                                {"label": dep, "value": dep}
                                for dep in (victimas['Dependencia de recepcion'].unique())
                            ],
                            value='172 - SECRETARÍA DE LAS MUJERES',
                            multi=False,
                            clearable=False,
                            #className="dropdown",
                        ),
                    ]
                ),
                #html.Div(
                    #children=[
                        #html.Div(children="Municipio", className="menu-title"),
                        #dcc.Dropdown(
                            #id="municipio-filter",
                            #options=[
                                #{"label": mun, "value": mun}
                                #for mun in victimas.municipiohechos.unique()
                            #],
                            #value=[mun for mun in victimas.municipiohechos.unique()],
                            #multi=True,
                            #clearable=True,
                            #searchable=True,
                            #className="dropdown",
                        #),
                    #],
                #),
                html.Div(
                    children=[
                        html.Div(
                            children="Date Range",
                            className="menu-title"
                            ),
                        dcc.DatePickerRange(
                            id="date-range",
                            min_date_allowed=victimas.fecha_recepcion.min(),
                            max_date_allowed=victimas.fecha_recepcion.max(),
                            start_date=victimas.fecha_recepcion.min(),
                            end_date=victimas.fecha_recepcion.max(),
                        ),
                    ]
                ),
            ],
            className="menu",
        ),
        dcc.Graph(
            id = 'map_1',  
        ),
        dcc.Graph(
            id = 'fig_1',  
        ),
        dcc.Graph(
            id = 'fig_2',  
        ),
    ]
)

In [12]:
@app.callback(
    [   Output('fig_1', "figure"),
        Output('map_1', "figure"),
        Output('fig_2', "figure"),
    ],
    [
        Input("dependencia-filter", "value"),
        #Input("municipio-filter", "value"),
        Input("date-range", "start_date"),
        Input("date-range", "end_date"),
    ]
)
def update_charts(dependencia, start_date, end_date):
    mask = (
        (victimas['Dependencia de recepcion'] == dependencia)
        #& (victimas['municipiohechos'].isin(municipio) ==True)
        & (victimas.fecha_recepcion >= start_date)
        & (victimas.fecha_recepcion <= end_date)
    )

    filtered_data = victimas.loc[mask, :]
    
    # Fig 1 - tipos de violencia
    tipos = filtered_data.iloc[:,[i for i in range(42,49)]]
    tipos = pd.DataFrame(tipos.sum())
    tipos = tipos.sort_values(0,ascending=False)
    y = tipos[0].values
    y_total = len(filtered_data)

    fig_1 = px.bar(x=tipos.index,
                y=tipos[0],
                text= np.round(y/y_total*100,2),
                labels = {'x': 'Tipo de violencia', "y":'Número de casos', 'text':'Porcentaje'},
                color=px.colors.qualitative.Prism[:7],
                color_discrete_map="identity",
                title='Gráfica 1. Casos de violencia hacia las mujeres por tipo de violencia'
                )
    fig_1.update_xaxes(type='category')
    fig_1.update_layout( xaxis_title=None)
    fig_1.update_traces(texttemplate='%{text} %')
    fig_1.update_traces(hovertemplate='Tipo de violencia=%{x}<br>Número de casos=%{y}<br>Porcentaje=%{text} %<extra></extra>')

    # Map 1 - casos x mun
    gdf = geopandas.GeoDataFrame(yuc, geometry='geometry_mun')
    
    mun_counts = pd.DataFrame(filtered_data.municipiohechos.value_counts())
    mun_counts.reset_index(inplace=True)
    mun_counts
    
    gdf = pd.DataFrame(gdf.merge(mun_counts,left_on='nom_mun',right_on='index',how='left'))
    gdf = geopandas.GeoDataFrame(gdf, geometry='geometry_mun')
    gdf = gdf.set_index('nom_mun')
    gdf = gdf.rename(columns={'municipiohechos':'Casos registrados'})
    gdf['Casos registrados log'] = np.log10(gdf['Casos registrados'] )
    gdf_crs = gdf.to_crs(pyproj.CRS.from_epsg(4326))
    #gdf_crs = gdf_crs.fillna(0)

    map_1 = px.choropleth(gdf_crs, geojson=gdf_crs.geometry_mun, 
                    locations=gdf_crs.index, color="Casos registrados log",
                    height=500,
                   color_continuous_scale="Viridis",
                    labels = {'nom_mun':'Municipio', "Casos registrados log":'log10'},
                    hover_data={"Casos registrados":True, "Casos registrados log":False},
                    title='Mapa 1. Casos Registrados por Municipio',
                   )
    map_1.update_geos(fitbounds="locations", visible=False)

    map_1.update_layout(xaxis=dict(domain=[0, 0.5]), yaxis=dict(domain=[0.25, 0.75]))

    map_1.add_annotation(
        # The arrow head will be 25% along the x axis, starting from the left
        x=0,
        # The arrow head will be 40% along the y axis, starting from the bottom
        y=0.98,
        text="<b>Temporalidad:</b> {}-{}<br><b>Total de casos:</b> {}".format(min(filtered_data.año_recepcion),max(filtered_data.año_recepcion),len(filtered_data)),
        showarrow=False,
        bordercolor="black",
        bgcolor="white",
        borderwidth=1.5,
        opacity=0.8
    )

    map_1.update(layout = dict(title=dict(x=0.5)))
    map_1.update_layout(
        margin={"r":0,"t":30,"l":10,"b":10},
        coloraxis_colorbar={
            'title':'Escala'})
    map_1.update_layout(title_y=1, title_x=0)
    map_1.update_layout(coloraxis_showscale=True)

    # fig 2 - estudios
    y_2 = filtered_data.escolaridad.value_counts().values
    y_total_2 = sum(y_2)
    colors = len(y_2)
    fig_2 = px.bar(x=filtered_data.escolaridad.value_counts().index,
                    y=filtered_data.escolaridad.value_counts().values,
                    text= np.round(y_2/y_total_2*100,2),
                    labels = {'x': 'Escolaridad', "y":'Número de casos', 'text':'Porcentaje'},
                    color=px.colors.qualitative.Prism[:colors],
                    color_discrete_map="identity",
                    title='Gráfica 2. Estudios Concluidos'
                    )
    fig_2.update_xaxes(type='category')
    fig_2.update_layout(xaxis_title=None)
    fig_2.update_traces(texttemplate='%{text} %')

    
    return fig_1,map_1,fig_2

In [13]:
"""mask = (
        (victimas['Dependencia de recepcion'] == '639 - TSJ-JUZGADO MIXTO CIVIL Y FAMILIAR DE KANASÍN')
        #& (victimas['municipiohechos'].isin(municipio) ==True)
        & (victimas.fecha_recepcion >= victimas.fecha_recepcion.min())
        & (victimas.fecha_recepcion <= victimas.fecha_recepcion.max())
    )

filtered_data = victimas.loc[mask, :]"""


"mask = (\n        (victimas['Dependencia de recepcion'] == '639 - TSJ-JUZGADO MIXTO CIVIL Y FAMILIAR DE KANASÍN')\n        #& (victimas['municipiohechos'].isin(municipio) ==True)\n        & (victimas.fecha_recepcion >= victimas.fecha_recepcion.min())\n        & (victimas.fecha_recepcion <= victimas.fecha_recepcion.max())\n    )\n\nfiltered_data = victimas.loc[mask, :]"

In [14]:
if __name__ == "__main__":
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Oct/2021 13:30:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/2021 13:30:09] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/2021 13:30:09] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/2021 13:30:09] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/2021 13:30:09] "GET /_dash-component-suites/dash/dcc/async-datepicker.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/2021 13:30:09] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/2021 13:30:09] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/2021 13:30:12] "POST /_dash-update-component HTTP/1.1" 200 -
